In [1]:
import pandas as pd
import numpy as np
import openpyxl as op

In [12]:
tab = pd.read_excel("results/tabmin_5840_-60_1.xlsx",sheet_name="Sheet1")

tab.rename(columns={0:-60, 1:1},inplace=True)

print(tab)

           -60       0.1
0     0.031854  0.031854
1     0.114561  0.114561
2     0.184065  0.184065
3     0.203126  0.203126
4     0.226382  0.226382
...        ...       ...
5834  0.001064  0.001064
5835  0.000000  0.000000
5836  0.000000  0.000000
5837  0.000000  0.000000
5838  0.000000  0.000000

[5839 rows x 2 columns]


In [14]:
# Exemple de DataFrame
data = {
    'A': range(20),
    'B': range(20, 40)
}
df = pd.DataFrame(data)
print(df)
# Définir l'intervalle pour les lignes vides
interval = 5

# Créer un nouveau DataFrame pour les résultats
new_index = np.sort(np.concatenate([df.index, np.arange(0.5, len(df), interval)]))
new_df = pd.DataFrame(index=new_index).reindex_like(df)

# Remplir le nouveau DataFrame avec les données existantes
new_df.update(df)

# Réindexer pour ajuster les lignes et rendre les index entiers
new_df.reset_index(drop=True, inplace=True)

print(new_df[4:12])

     A   B
0    0  20
1    1  21
2    2  22
3    3  23
4    4  24
5    5  25
6    6  26
7    7  27
8    8  28
9    9  29
10  10  30
11  11  31
12  12  32
13  13  33
14  14  34
15  15  35
16  16  36
17  17  37
18  18  38
19  19  39
       A     B
4    4.0  24.0
5    5.0  25.0
6    6.0  26.0
7    7.0  27.0
8    8.0  28.0
9    9.0  29.0
10  10.0  30.0
11  11.0  31.0
